# Gallery Example: M/M/1-PS Reentrant Queue

This example demonstrates an M/M/1 queueing system with Processor Sharing and reentrant routing:
- **Arrivals**: Exponential (Poisson process)
- **Service**: Exponential
- **Servers**: 1 server
- **Scheduling**: Processor Sharing (PS)
- **Reentrant**: Jobs switch class and pass through the queue again

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mm1_ps_reentrant(): Network {
    """Create M/M/1-PS-Reentrant queueing model"""
    val model = Network("M/M/1")
// Block 1: nodes
    val source = Source(model, "Source")
    val queue = Queue(model, "Queue", SchedStrategy.PS)  # Processor Sharing
    val sink = Sink(model, "Sink")
// Block 2: classes
    val oclass1 = OpenClass(model, "Class1")
    val oclass2 = OpenClass(model, "Class2")
// Exponential arrivals for Class1, Class2 is disabled at source
    source.setArrival(oclass1, Exp(1))
    source.setArrival(oclass2, Disabled())
// Exponential service for both classes
    queue.setService(oclass1, Exp(2))
    queue.setService(oclass2, Exp(3))
// Block 3: topology with class switching (reentrant)
    val P = model.initRoutingMatrix()
    P.addRoute(oclass1, source, queue, 1.0)
    P.addClassSwitch(oclass1, oclass2, queue, queue, 1.0)
    P.addRoute(oclass2, queue, sink, 1.0)
    model.link(P)
    
    return model
// Create the model
val model = gallery_mm1_ps_reentrant()

## About Processor Sharing with Reentrant Routing

**Processor Sharing (PS)** scheduling:
- All jobs share the server capacity equally
- Each of n jobs receives 1/n of the server capacity
- Fair scheduling independent of service time

**With reentrant routing**:
- Jobs pass through the queue twice (as Class1, then Class2)
- Under PS, both classes share capacity simultaneously
- The mean response time depends on total work in the system

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)